In [ ]:
# Uncomment the following block to install required libraries
# """
# !pip install langchain chromadb sentence-transformers
# !pip install  openai tiktoken
# !pip install jq
# !pip install faiss
# !pip install pymilvus
# """

In [1]:

!git config --global user.email "xz3761@nyu.edu"
!git config --global user.name "skylarzhao1"


In [ ]:
!git -C ColBERT/ pull || git clone https://github.com/stanford-futuredata/ColBERT.git
import sys; sys.path.insert(0, 'ColBERT/')

Already up to date.


In [ ]:
try: # When on google Colab, let's install all dependencies with pip.
    import google.colab
    !pip install -U pip
    !pip install -e ColBERT/['faiss-gpu','torch']
except Exception:
  import sys; sys.path.insert(0, 'ColBERT/')
  try:
    from colbert import Indexer, Searcher
  except Exception:
    print("If you're running outside Colab, please make sure you install ColBERT in conda following the instructions in our README. You can also install (as above) with pip but it may install slower or less stable faiss or torch dependencies. Conda is recommended.")
    assert False

Obtaining file:///content/ColBERT
  Preparing metadata (setup.py) ... done
  Using cached bitarray-3.4.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
  Using cached ninja-1.11.1.4-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.0 kB)
INFO: pip is looking at multiple versions of colbert-ai[faiss-gpu,torch] to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement faiss-gpu>=1.7.0; extra == "faiss-gpu" (from colbert-ai[faiss-gpu,torch]) (from versions: none)
ERROR: No matching distribution found for faiss-gpu>=1.7.0; extra == "faiss-gpu"


In [ ]:
!pip install ujson
import colbert
from colbert import Indexer, Searcher
from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection


In [ ]:
!pip install llama-index-core transformers torch
!pip install llama-index-postprocessor-colbert-rerank
!pip install llama-index-embeddings-huggingface

In [ ]:
!pip install -qU bitsandbytes transformers accelerate

In [ ]:
from transformers import BitsAndBytesConfig  # 新增这行
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


- Setting the API key of HuggingFace to load the model  

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']='your api token'

* Load the PubMed articles from the JSON file. To prepare the JSON file, please refer to the script `download_pubmed.py`

In [ ]:
!pip install -U langchain langchain-community
!pip install jq
#加载 JSON 文件里的 PubMed 文章摘要，并封装成 LangChain 能处理的 Document 格式，方便后续做 chunking、embedding 和 RAG 检索
from langchain.document_loaders import JSONLoader

def metadata_func(record: dict, metadata: dict) -> dict:
    # Define the metadata extraction function.
    metadata["year"] = record.get("pub_date").get('year')
    metadata["month"] = record.get("pub_date").get('month')
    metadata["day"] = record.get("pub_date").get('day')
    metadata["title"] = record.get("article_title")

    return metadata

loader = JSONLoader(
    file_path='/content/drive/MyDrive/finalproject/pubmed_article.json',
    jq_schema='.[]',
    content_key='article_abstract',
    metadata_func=metadata_func)
data = loader.load()
print(f"{len(data)} pubmed articles are loaded!")
data[1]

2549 pubmed articles are loaded!


Document(metadata={'source': '/content/drive/MyDrive/finalproject/pubmed_article.json', 'seq_num': 2, 'year': '2024', 'month': '12', 'day': '17', 'title': 'Hepatolithiasis pathogenesis update.'}, page_content="Hepatolithiasis is prevalent in East Asian countries and not common in Western countries. In recent years, because of the increased number of immigrants from East Asia in Western countries, hepatolithiasis has gradually become a global problem. Although current surgical interventions for hepatolithiasis boast a high rate of stone clearance, the persistent challenges of the disease's refractory nature and high recurrence rate continue to complicate its treatment. Therefore, understanding its underlying pathogenesis is meaningful for effective treatment. In this review, we discuss the common risk factors: infection, cholangitis, environmental factors and diet habits, abnormal bile components, anatomical abnormalities, and bile stasis, and summarize the relevant mechanisms.")

- Chunk abstracts into small text passages for efficient retrieval and LLM context length

In [ ]:
!pip install tiktoken
#只对page content 做chunking
from langchain.text_splitter import TokenTextSplitter,CharacterTextSplitter
text_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=64)
chunks = text_splitter.split_documents(data)
print(f"{len(data)} pubmed articles are converted to {len(chunks)} text fragments!")
chunks[0]

2549 pubmed articles are converted to 4328 text fragments!


Document(metadata={'source': '/content/drive/MyDrive/finalproject/pubmed_article.json', 'seq_num': 1, 'year': '2025', 'month': '04', 'day': '03', 'title': 'CORRIGENDUM.'}, page_content='[This corrects the article DOI: 10.48101/ujms.v129.10741.].')

- Load the embedding model. The following code defines two options for loading the model:
    - **Option a:** Using SentenceTransformerEmbeddings framework to load their most performing model `all-mpnet-base-v2`
    - **Option b:** Using HuggingFaceEmbeddings hub to load the popular model `e5-large-unsupervised`

In [ ]:
# Option a: using all-mpnet from SentenceTransformer
# from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
# embeddings = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

# # Option b: using e5-large-unspupervised from huggingface
# from langchain.embeddings import HuggingFaceEmbeddings
# modelPath = "intfloat/e5-large-unsupervised"
# embeddings = HuggingFaceEmbeddings(
#   model_name = modelPath,
#   model_kwargs = {'device':'cuda'},
#   encode_kwargs={'normalize_embeddings':False}
# )



from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(
    model_name="intfloat/e5-large-v2",
    model_kwargs={"device": "cuda"}
)


<ipython-input-21-546ed2265880>:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/67.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
!pip install faiss-gpu-cu12


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 MB 118.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 148.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [faiss-gpu-cu12]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:

!pip install chromadb


# Option a: Using chroma database
from langchain.vectorstores import Chroma
# db = Chroma.from_documents(chunks, embeddings)
# 用正确的 1024 维模型加载
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-large-en-v1.5")


db = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory="./chroma_db"
)


'''
# Option b: Using Milvus database
# To run the following code, you should have a milvus instance up and running
# Follow the instructions in the following the link: https://milvus.io/docs/install_standalone-docker.md
from langchain.vectorstores import Milvus
db = Milvus.from_documents(
    chunks,
    embeddings,
    connection_args={"host": "127.0.0.1", "port": "19530"},
)
'''

# # Using faiss index
# from langchain.vectorstores import FAISS
# db = FAISS.from_documents(chunks, embeddings)

'\n# Option b: Using Milvus database\n# To run the following code, you should have a milvus instance up and running\n# Follow the instructions in the following the link: https://milvus.io/docs/install_standalone-docker.md\nfrom langchain.vectorstores import Milvus\ndb = Milvus.from_documents(\n    chunks,\n    embeddings,\n    connection_args={"host": "127.0.0.1", "port": "19530"},\n)\n'

- Build the vector databse (VDB) to index the text chunks and their corresponsding vectors. We also define three options to define the VDB:
    - **Option a:** Using chromaDB
    - **Option b:** Using Milvus
    - **Option c:** Using FAISS index

#TODO Add definition and comparison between the two options

- Load pre-trained Mistral 7B

In [ ]:

from huggingface_hub import notebook_login
notebook_login()

from transformers import BitsAndBytesConfig


from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline
from langchain import HuggingFacePipeline

#model_id = "mistralai/Mistral-7B-v0.1"

# 1. 配置4-bit量化
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)


model_id = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=False, device_map='auto')

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=128)
llm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0, "max_length": 1024}
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


- Define the RAG pipeline using LangChain. The LLM's answer highly depends on the prompt template, that's why we tested three different prompts. The one giving the best answer as PROMPT2.

#TODO: Add explanation about the three prompts

In [ ]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import time

# PROMPT 1
PROMPT_TEMPLATE_1 = """Answer the question based only on the following context:
{context}
You are allowed to rephrase the answer based on the context.
Question: {question}
"""
PROMPT1 = PromptTemplate.from_template(PROMPT_TEMPLATE_1)

# # PROMPT 2
# PROMPT_TEMPLATE_2="Your are a medical assistant for question-answering tasks. Answer the Question using the provided Contex only. Your answer should be in your own words and be no longer than 128 words. \n\n Context: {context} \n\n Question: {question} \n\n Answer:"
# PROMPT2 = PromptTemplate.from_template(PROMPT_TEMPLATE_2)

# 1. 修正Prompt
PROMPT_TEMPLATE_2 = """Answer based ONLY on:
Context: {context}
Question: {question}
Concise medical answer (max 100 words):"""
PROMPT2 = PromptTemplate.from_template(PROMPT_TEMPLATE_2)


# # 3. 执行查询
# start_time = time.time()
# result = qa_chain.invoke({"query": "Alzheimer's treatments"})
# print(f"Time: {time.time() - start_time:.2f}s")
# print(result["result"])
# PROMPT 3
from langchain import hub
PROMPT3 = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

# RAG pipeline
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(k=4),
    chain_type_kwargs={"prompt": PROMPT2},
    return_source_documents=True
)


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


NameError: name 'llm' is not defined

- Run one sample query `"What are the safest cryopreservation methods?"

In [ ]:
start_time = time.time()
# query = "What are the safest cryopreservation methods?"
query="What are the recent advancements in the treatment of Alzheimer’s disease?"
result = qa_chain({"query": query})
print(f"\n--- {time.time() - start_time} seconds ---")

<ipython-input-30-fd0571a3e187>:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": query})
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [ ]:
retrieved_docs = result["source_documents"]

print(f"\n 共检索到 {len(retrieved_docs)} 个文献段落（chunk）：\n")

for i, doc in enumerate(retrieved_docs):
    print(f" Chunk {i+1}:")
    print(f" Title: {doc.metadata.get('title')}")
    print(f" Content Preview: {doc.page_content[:200]}...")  # 只显示前200个字
    print("-" * 80)


In [ ]:
print(result['result'].strip())
titles = ['\t-'+doc.metadata['title'] for doc in result['source_documents']]
print("\n\nThe provided answer is based on the following PubMed articles:\t")
print("\n".join(set(titles)))

- Get the answer to the sample query from the LLM only

In [ ]:
# Define the langchain pipeline for llm only
from langchain.prompts import PromptTemplate
PROMPT_TEMPLATE ="""Answer the given Question only. Your answer should be in your own words and be no longer than 100 words. \n\n Question: {question} \n\n
Answer:
"""
PROMPT = PromptTemplate.from_template(PROMPT_TEMPLATE)
llm_chain = PROMPT | llm
start_time = time.time()
result = llm_chain.invoke({"question": query})
print(f"\n--- {time.time() - start_time} seconds ---")
print(result)

In [ ]:
import torch
print(torch.cuda.is_available())


#Indexing


nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 128 # truncate passages at 300 tokens
max_id = 20
checkpoint = "colbert-ir/colbertv2.0"


collection = [chunk.page_content for chunk in chunks[:max_id]]  # 限制了 max_id=2400，所以截取前 2400 条


index_name = 'pubmed_index_small'

with Run().context(RunConfig(nranks=1, experiment="/content/drive/MyDrive/finalproject/exp_pubmed")):

    config = ColBERTConfig(doc_maxlen=128, nbits=2, kmeans_niters=2)
    indexer = Indexer(checkpoint=checkpoint, config=config)
    indexer.index(name=index_name, collection=collection, overwrite=True)

True


[May 07, 18:56:16] #> Note: Output directory /content/drive/MyDrive/finalproject/exp_pubmed/indexes/pubmed_index_small already exists


[May 07, 18:56:16] #> Will delete 1 files already at /content/drive/MyDrive/finalproject/exp_pubmed/indexes/pubmed_index_small in 20 seconds...
#> Starting...


KeyboardInterrupt: 

In [ ]:
!pip install -U sentence-transformers
from sentence_transformers import CrossEncoder


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings



# 加载 BGE Reranker 模型
rerank_model_name = "BAAI/bge-reranker-base"
tokenizer = AutoTokenizer.from_pretrained(rerank_model_name)
model = AutoModelForSequenceClassification.from_pretrained(rerank_model_name)
model.eval().to("cuda" if torch.cuda.is_available() else "cpu")
tokenizer.pad_token = tokenizer.eos_token  #
def rerank_bge(query, documents, top_k=3):
    scores = []
    for doc in documents:
        inputs = tokenizer(
            query, doc.page_content,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        ).to(model.device)

        with torch.no_grad():
            logits = model(**inputs).logits
            score = logits.squeeze().item()  #

        scores.append((doc, score))

    reranked = sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]
    for i, (doc, score) in enumerate(reranked):
        print(f"Reranked #{i+1} | Score: {score:.4f}")
        print(f"Title: {doc.metadata.get('title')}")
        print(f"Content: {doc.page_content[:300]}...\n")
    return [doc for doc, score in reranked]


# query = "What are the recent advancements in the treatment of Alzheimer’s disease?"
# retrieved_docs = db.as_retriever(k=8).get_relevant_documents(query)

# # rerank
# reranked_docs = rerank_bge(query, retrieved_docs, top_k=7)


# print(f"\n Query: {query}")
# print(f" Top {len(reranked_docs)} reranked results:\n")
# for i, doc in enumerate(reranked_docs):
#     print(f"Reranked #{i+1}:")
#     print(f"Title: {doc.metadata.get('title')}")
#     print(f"Content: {doc.page_content[:300]}...\n")


In [ ]:
query = "What are the recent advancements in the treatment of Alzheimer’s disease?"
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE_2 = """Answer based ONLY on:
Context: {context}
Question: {question}
Concise medical answer:"""
PROMPT2 = PromptTemplate.from_template(PROMPT_TEMPLATE_2)

# Step 1: 检索
retrieved_docs = db.as_retriever(k=7).get_relevant_documents(query)

# Step 2: rerank
reranked_docs = rerank_bge(query, retrieved_docs, top_k=4)

# Step 3: 拼接上下文
context = "\n\n".join([doc.page_content for doc in reranked_docs])

# Step 4: 构造 Prompt
prompt = PROMPT2.format(context=context, question=query)

# Step 5: LLM 推理
response = llm.invoke(prompt)
print(" Final Answer:\n", response)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Reranked #1 | Score: 0.3136
Title: Polydatin attenuates Alzheimer's disease induced by aluminum chloride in rats: evidence for its antioxidant and anti-inflammatory effects.
Content: Considering the complex pathophysiological mechanisms behind Alzheimer's disease (AD), a few drugs for managing related cognitive symptoms have been approved. The phytochemical resveratrol has shown promising anti-inflammatory and antioxidant effects in AD, but it has low bioavailability. Chemical m...

Reranked #2 | Score: -1.3051
Title: Effects of human concurrent aerobic and resistance training on cognitive health: A systematic review with meta-analysis.
Content: The rising prevalence of cognitive decline and neurodegenerative diseases, projected to affect 150 million individuals by 2050, highlights the urgent need to enhance neurocognitive health. While both aerobic and resistance training are recognized as effective strategies, their combined effects on co...

Reranked #3 | Score: -5.2209
Title: Nanom